In [1]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
import sqlite3

In [2]:
from sqlalchemy import Table, Column, Integer, Numeric, String, DateTime, Boolean
from sqlalchemy import ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import sessionmaker

In [3]:
from datetime import datetime


In [4]:
engine = create_engine('sqlite:///cookies.db', echo=True)
engine

Engine(sqlite:///cookies.db)

In [5]:
Base = declarative_base()

In [6]:
metadata = MetaData()

In [7]:
# t = Base.metadata.tables['cookies']
# Base.metadata.remove(t)

class Cookie(Base):
    __tablename__ = "cookies"
    
    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer)
    unit_cost = Column(Numeric(12, 2))
    
#     def __repr__(self):
#         return "Cookie(cookie_name='{self.cookie_name}'".format(self=self)

In [8]:
# t = Base.metadata.tables['users']
# Base.metadata.remove(t)

class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer, primary_key=True)
    username = Column(Integer, nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)
    
#     def __repr__(self):
#         return "User(username='{self.username}')".format(self=self)


In [9]:
# t = Base.metadata.tables['orders']
# Base.metadata.remove(t)

class Order(Base):
    __tablename__ = 'orders'
    
    order_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.user_id'))
    shipped = Column(Boolean, default=False)
    
    user = relationship('User', backref=backref('orders', order_by=order_id))

#     def __repr__(self):
#         return "Order(user_id='{self.user_id}', shipped='{self.shipped}')".format(self=self)

In [10]:
# t = Base.metadata.tables['orders']
# Base.metadata.remove(t)

class LineItem(Base):
    __tablename__ = 'line_items'
    
    line_item_id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey('orders.order_id'))
    cookie_id = Column(Integer, ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer)
    extended_cost = Column(Numeric(12, 2))
    
    order = relationship('Order', backref=backref('line_items', order_by=line_item_id))
    cookie = relationship('Cookie', uselist=False)
    
#     def __repr__(self):
#         return "LineItems(order_id='{self.order_id}', cookie_id='{self.cookie_id}', quantity='{self.quantity}')".format(self=self)

In [11]:
for t in Base.metadata.tables:
    print(t)
    print("---")

cookies
---
users
---
orders
---
line_items
---


In [14]:
Base.metadata.drop_all(engine)

2022-02-04 01:33:20,299 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-04 01:33:20,300 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2022-02-04 01:33:20,301 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:20,302 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-02-04 01:33:20,303 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:20,304 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2022-02-04 01:33:20,305 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:20,306 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("line_items")
2022-02-04 01:33:20,306 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:20,307 INFO sqlalchemy.engine.Engine 
DROP TABLE line_items
2022-02-04 01:33:20,308 INFO sqlalchemy.engine.Engine [no key 0.00078s] ()
2022-02-04 01:33:20,310 INFO sqlalchemy.engine.Engine 
DROP TABLE orders
2022-02-04 01:33:20,311 INFO sqlalchemy.engine.Engine [no key 0.00076s] ()
2022-02-0

In [15]:
Base.metadata.create_all(engine)

### 真實中斷 DB 連線
# engine.dispose()

2022-02-04 01:33:30,418 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-04 01:33:30,419 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2022-02-04 01:33:30,421 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:30,422 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cookies")
2022-02-04 01:33:30,423 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:30,424 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-02-04 01:33:30,424 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:30,425 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-02-04 01:33:30,426 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:30,427 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2022-02-04 01:33:30,428 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-04 01:33:30,428 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2022-02-04 01:33:30,429 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-

---

## Session

In [16]:
Session = sessionmaker(bind=engine)
session = Session()
session

In [17]:
cc_cookie = Cookie(cookie_name='chocolate chip',
                   cookie_recipe_url='http://some.aweso.me/cookie/recipe.html',
                   cookie_sku='CC01',
                   quantity=12,
                   unit_cost=0.5)
session.add(cc_cookie)
session.commit()

2022-02-04 01:34:11,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-04 01:34:11,150 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2022-02-04 01:34:11,151 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, 0.5)
2022-02-04 01:34:11,153 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
dcc = Cookie(cookie_name='dark chocolate chip',
             cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
             cookie_sku='CC02',
             quantity=1,
             unit_cost=.75)
mol = Cookie(cookie_name='molasses',
             cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html',
             cookie_sku='MOL01',
             quantity=1,
             unit_cost=.8)
session.add(dcc)
session.add(mol)
session.flush()
print(dcc.cookie_id)
print(mol.cookie_id)

2022-02-04 01:34:46,988 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-04 01:34:46,990 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2022-02-04 01:34:46,991 INFO sqlalchemy.engine.Engine [cached since 35.84s ago] ('dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75)
2022-02-04 01:34:46,992 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2022-02-04 01:34:46,993 INFO sqlalchemy.engine.Engine [cached since 35.84s ago] ('molasses', 'http://some.aweso.me/cookie/recipe_molasses.html', 'MOL01', 1, 0.8)
2
3


In [20]:
c1 = Cookie(cookie_name='peanut butter',
            cookie_recipe_url='http://some.aweso.me/cookie/peanut.html',
            cookie_sku='PB01',
            quantity=24,
            unit_cost=.25)
c2 = Cookie(cookie_name='oatmeal raisin',
            cookie_recipe_url='http://some.aweso.me/cookie/raisin.html',
            cookie_sku='EWW01',
            quantity=100,
            unit_cost=1.00)
session.bulk_save_objects([c1, c2])
session.commit()

2022-02-04 01:39:25,919 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2022-02-04 01:39:25,921 INFO sqlalchemy.engine.Engine [generated in 0.00160s] (('peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, 0.25), ('oatmeal raisin', 'http://some.aweso.me/cookie/raisin.html', 'EWW01', 100, 1.0))
2022-02-04 01:39:25,922 INFO sqlalchemy.engine.Engine COMMIT
